# Assignment 1: NYC Taxi Data

In [2]:
from pyspark.sql import SparkSession

In [3]:
# Create a local spark session
spark = SparkSession.builder \
        .appName('nyc-taxi-sql') \
        .getOrCreate()

In [4]:
# Read parquet file
df = spark.read.load("./output")

In [5]:
df.createOrReplaceTempView("nyc_taxi_data_2017_18")

## Business Questions

### Q1.a. For each year and month: What was the total number of trips?

In [5]:
spark.sql("""
SELECT year
,month
,COUNT(*) AS number_of_trips
FROM nyc_taxi_data_2017_18
GROUP BY year
,month
ORDER BY year, month
""").show(24)

+----+-----+---------------+
|year|month|number_of_trips|
+----+-----+---------------+
|2017|    1|       10759055|
|2017|    2|       10170592|
|2017|    3|       11429334|
|2017|    4|       11104411|
|2017|    5|       11139331|
|2017|    6|       10612182|
|2017|    7|        9483901|
|2017|    8|        9271000|
|2017|    9|        9808837|
|2017|   10|       10673291|
|2017|   11|       10137773|
|2017|   12|       10393990|
|2018|    1|        9535011|
|2018|    2|        9244198|
|2018|    3|       10246590|
|2018|    4|       10086530|
|2018|    5|       10002146|
|2018|    6|        9433947|
|2018|    7|        8516297|
|2018|    8|        8497622|
|2018|    9|        8688822|
|2018|   10|        9508435|
|2018|   11|        8781808|
|2018|   12|        8837417|
+----+-----+---------------+



### Q1.b. For each year and month: Which weekday had the most trips?

In [6]:
spark.sql("""
SELECT year
    ,month
    ,pickup_weekday
    ,total_trips
FROM (SELECT year
        ,month
        ,DATE_FORMAT(pickup_datetime, "EEEE") AS pickup_weekday
        ,COUNT(*) AS total_trips
        ,ROW_NUMBER() OVER (PARTITION BY year,month ORDER BY COUNT(*) DESC) AS row_num
    FROM nyc_taxi_data_2017_18
    GROUP BY year
        ,month
        ,pickup_weekday
    )
WHERE row_num = 1
ORDER BY year
    ,month
""").show(24)

+----+-----+--------------+-----------+
|year|month|pickup_weekday|total_trips|
+----+-----+--------------+-----------+
|2017|    1|       Tuesday|    1698667|
|2017|    2|      Saturday|    1613115|
|2017|    3|        Friday|    2030231|
|2017|    4|      Saturday|    1965173|
|2017|    5|     Wednesday|    1857762|
|2017|    6|      Thursday|    1852070|
|2017|    7|      Saturday|    1526780|
|2017|    8|      Thursday|    1603485|
|2017|    9|        Friday|    1721426|
|2017|   10|       Tuesday|    1673294|
|2017|   11|     Wednesday|    1740282|
|2017|   12|        Friday|    1827482|
|2018|    1|     Wednesday|    1624943|
|2018|    2|        Friday|    1462063|
|2018|    3|        Friday|    1808358|
|2018|    4|        Monday|    1520937|
|2018|    5|      Thursday|    1741622|
|2018|    6|        Friday|    1641972|
|2018|    7|       Tuesday|    1453861|
|2018|    8|     Wednesday|    1485514|
|2018|    9|      Saturday|    1469617|
|2018|   10|     Wednesday|    1572695|


### Q1.c. For each year and month: What was the average number of passengers?

In [8]:
spark.sql("""
SELECT year
    ,month
    ,AVG(passenger_count) AS avg_passengers_per_trip
FROM nyc_taxi_data_2017_18
GROUP BY year
    ,month
ORDER BY year
    ,month
""").show(24)

+----+-----+-----------------------+
|year|month|avg_passengers_per_trip|
+----+-----+-----------------------+
|2017|    1|     1.6035315369240142|
|2017|    2|     1.5991538152351408|
|2017|    3|     1.5928098697614401|
|2017|    4|     1.6020269782881775|
|2017|    5|     1.5956274214313229|
|2017|    6|     1.5996936351072757|
|2017|    7|     1.6155018910467327|
|2017|    8|     1.6097582785028584|
|2017|    9|     1.6050604164387685|
|2017|   10|     1.5993137449358403|
|2017|   11|     1.5957080514625845|
|2017|   12|     1.6152579519510795|
|2018|    1|     1.5930920268471636|
|2018|    2|     1.5828088061289902|
|2018|    3|     1.5889266575514391|
|2018|    4|     1.5892638003356951|
|2018|    5|     1.5853707794307341|
|2018|    6|      1.586684025254753|
|2018|    7|     1.5937331683007299|
|2018|    8|     1.5902972619869418|
|2018|    9|     1.5784291587513244|
|2018|   10|     1.5640660108629865|
|2018|   11|     1.5717419465331057|
|2018|   12|     1.5884660642357376|
+

### Q1.d. For each year and month: What was the average amount paid per trip (total_amount)?

In [10]:
spark.sql("""
SELECT year
    ,month
    ,AVG(total_amount) AS avg_total_amount_per_trip
FROM nyc_taxi_data_2017_18
GROUP BY year
    ,month
ORDER BY year
    ,month
""").show(24)

+----+-----+-------------------------+
|year|month|avg_total_amount_per_trip|
+----+-----+-------------------------+
|2017|    1|        15.30173950320955|
|2017|    2|       15.470210002462021|
|2017|    3|        16.00380195564449|
|2017|    4|        16.10720418329613|
|2017|    5|       16.560673703888867|
|2017|    6|        16.47228243213232|
|2017|    7|         16.2144495542399|
|2017|    8|        16.30985790548921|
|2017|    9|       16.515653416610583|
|2017|   10|       16.576218103559412|
|2017|   11|        16.32591823573312|
|2017|   12|       16.032728606820424|
|2018|    1|        15.38746873304433|
|2018|    2|       15.387113757366647|
|2018|    3|       15.901031332181095|
|2018|    4|       16.261353141312924|
|2018|    5|       16.755073060361443|
|2018|    6|       16.653265007935776|
|2018|    7|       16.569750328297385|
|2018|    8|       16.601714209171014|
|2018|    9|       16.834233158417458|
|2018|   10|       16.933848321180925|
|2018|   11|       16.818

### Q1.d. For each year and month: What was the average amount paid per passenger (total_amount)?

In [11]:
spark.sql("""
SELECT year
    ,month
    ,AVG(total_amount / passenger_count) AS avg_total_amount_per_passenger
FROM nyc_taxi_data_2017_18
GROUP BY year
    ,month
ORDER BY year
    ,month
""").show(24)

+----+-----+------------------------------+
|year|month|avg_total_amount_per_passenger|
+----+-----+------------------------------+
|2017|    1|            12.646149725201287|
|2017|    2|            12.764816082660923|
|2017|    3|            13.245625458581475|
|2017|    4|            13.265101183416846|
|2017|    5|            13.651144429622907|
|2017|    6|            13.604502486696209|
|2017|    7|            13.287119010142096|
|2017|    8|             13.39625704158913|
|2017|    9|            13.590143123322886|
|2017|   10|             13.68152108161739|
|2017|   11|            13.480742929428729|
|2017|   12|            13.117433840563063|
|2018|    1|            12.735796628332748|
|2018|    2|            12.776375687262949|
|2018|    3|            13.154252048935492|
|2018|    4|            13.445791517382709|
|2018|    5|             13.87456607832239|
|2018|    6|            13.777108741419482|
|2018|    7|            13.682049332549617|
|2018|    8|            13.71762

### Q2.a. For each taxi colour (yellow and green): What was the average, median, minimum and maximum trip duration in seconds?

In [16]:
spark.sql("""
SELECT taxi_type
    ,AVG(trip_duration_seconds) AS avg_trip_duration_seconds
    ,PERCENTILE(trip_duration_seconds, 0.5) AS median_trip_duration_seconds
    ,MIN(trip_duration_seconds) AS min_trip_duration_seconds
    ,MAX(trip_duration_seconds) AS max_trip_duration_seconds
FROM nyc_taxi_data_2017_18
GROUP BY taxi_type
""").show(2)

+---------+-------------------------+----------------------------+-------------------------+-------------------------+
|taxi_type|avg_trip_duration_seconds|median_trip_duration_seconds|min_trip_duration_seconds|max_trip_duration_seconds|
+---------+-------------------------+----------------------------+-------------------------+-------------------------+
|    green|       1266.2004888441165|                       627.0|                        1|                   202989|
|   yellow|       1022.0828914491414|                       670.0|                        1|                 45466304|
+---------+-------------------------+----------------------------+-------------------------+-------------------------+



### Q2.b. For each taxi colour (yellow and green): What was the average, median, minimum and maximum trip distance in km?

In [5]:
spark.sql("""
SELECT taxi_type
    ,AVG(trip_distance_km) AS avg_trip_distance_km
    ,PERCENTILE(trip_distance_km, 0.5) AS median_trip_distance_km
    ,MIN(trip_distance_km) AS min_trip_distance_km
    ,MAX(trip_distance_km) AS max_trip_distance_km
FROM nyc_taxi_data_2017_18
GROUP BY taxi_type
""").show(2)

+---------+--------------------+-----------------------+--------------------+--------------------+
|taxi_type|avg_trip_distance_km|median_trip_distance_km|min_trip_distance_km|max_trip_distance_km|
+---------+--------------------+-----------------------+--------------------+--------------------+
|   yellow|   4.728245869247112|     2.6232241999999997|                 0.0|         4059.157815|
+---------+--------------------+-----------------------+--------------------+--------------------+



### Q2.c. For each taxi colour (yellow and green): What was the average, median, minimum and maximum speed in km per hour?

In [6]:
spark.sql("""
SELECT taxi_type
    ,AVG(trip_distance_km/(trip_duration_seconds / 3600)) AS avg_km_per_hour
    ,PERCENTILE(trip_distance_km/(trip_duration_seconds / 3600), 0.5) AS median_km_per_hour
    ,MIN(trip_distance_km/(trip_duration_seconds / 3600)) AS min_km_per_hour
    ,MAX(trip_distance_km/(trip_duration_seconds / 3600)) AS max_km_per_hour
FROM nyc_taxi_data_2017_18
GROUP BY taxi_type
""").show(2)

+---------+-----------------+------------------+---------------+---------------+
|taxi_type|  avg_km_per_hour|median_km_per_hour|min_km_per_hour|max_km_per_hour|
+---------+-----------------+------------------+---------------+---------------+
|    green|22.64211146324986| 17.79052218181818|            0.0|    194955.4476|
+---------+-----------------+------------------+---------------+---------------+



### Q2.d. For each taxi colour (yellow and green): What was the percentage of trips where the driver received tips?

In [13]:
spark.sql("""
SELECT ((SELECT COUNT(*) FROM nyc_taxi_data_2017_18 WHERE tip_amount > 0) / COUNT(*)) * 100 AS pct_trips_with_tip
FROM nyc_taxi_data_2017_18
""").show(1)

+------------------+
|pct_trips_with_tip|
+------------------+
| 63.05336311357655|
+------------------+



### Q3. For trips where the driver received tips, What was the percentage where the driver received tips of at least $10.

In [14]:
spark.sql("""
SELECT ((SELECT COUNT(*) FROM nyc_taxi_data_2017_18 WHERE tip_amount >= 10) / COUNT(*)) * 100 AS pct_trips_top_gt_10
FROM nyc_taxi_data_2017_18
""").show(1)

+-------------------+
|pct_trips_top_gt_10|
+-------------------+
| 2.1053562129901136|
+-------------------+



### Q4.a. For each duration bin calculate: Average speed (km per hour)
Bins are Under 5 Mins, From 5 mins to 10 mins, From 10 mins to 20 mins, From 20 mins to 30 mins, At least 30 mins:

In [7]:
spark.sql("""
SELECT trip_duration_category
    ,AVG(trip_distance_km / (trip_duration_seconds / 3600)) AS avg_km_per_hour
FROM nyc_taxi_data_2017_18
GROUP BY trip_duration_category
""").show(5)

+----------------------+------------------+
|trip_duration_category|   avg_km_per_hour|
+----------------------+------------------+
|         Above 30 mins|21.521682982544082|
|            10-20 mins| 20.07051347804941|
|             5-10 mins|17.981705341505787|
|            20-30 mins| 21.78188930509953|
|          Under 5 mins| 37.06728243111635|
+----------------------+------------------+



### Q4.b. For each duration bin calculate: Average distance per dollar (km per $)
Bins are Under 5 Mins, From 5 mins to 10 mins, From 10 mins to 20 mins, From 20 mins to 30 mins, At least 30 mins.

Assuming total US dollars received for journey, which includes tips, special fees and taxes

In [11]:
spark.sql("""
SELECT trip_duration_category
    ,AVG(trip_distance_km / total_amount) AS avg_distance_per_dollar
FROM nyc_taxi_data_2017_18
GROUP BY trip_duration_category
""").show(5)

+----------------------+-----------------------+
|trip_duration_category|avg_distance_per_dollar|
+----------------------+-----------------------+
|         Above 30 mins|    0.40398652813664593|
|            10-20 mins|     0.3094178375755084|
|             5-10 mins|    0.24283247313513143|
|            20-30 mins|     0.3585894731795521|
|          Under 5 mins|    0.17535589774042296|
+----------------------+-----------------------+



### Q5. Which duration bin will you advise a taxi driver to target to maximise his income?